## 라벨 데이터의 클래스, 메뉴 정보를 매핑하여 새로운 json 파일 생성

### 0. 라이브러리 설치

In [1]:
import pandas as pd
import numpy as np
import os
import zipfile
import glob
import shutil
import json

### 1. 경로 설정

### 2. 데이터 불러오기

In [20]:
path = './'
data = os.path.join(path + 'label_data.zip')
data

'./label_data.zip'

### 3. json 파일에서 필요한 정보를 추출하여 새로운 파일에 저장

* 새로운 파일, 변수 선언

In [24]:
# 새로운 json 파일 생성
with open('menu_info.json', 'w', encoding='utf-8') as f:
  json.dump({}, f, indent=4)

# 메뉴 정보를 저장할 리스트, 현재까지 매핑된 메뉴명을 저장할 튜플과와 max_class_id 변수
menu_info = []
visited_data = set()
max_class_id = -1

# 중복된 메뉴 개수(그냥 궁금해서)
overlapping = 0

* 정보 추출 후 타겟 json 파일로 옮기는 함수

In [29]:
def change_json(input_file):
  global menu_info, visited_data, max_class_id, overlapping

  # 입력 파일 읽기
  data = json.load(input_file)

  for annotation in data.get("annotations", []):
    menu_in = annotation.get("menu_information", {})
    menu_name_ko = menu_in.get("ko")

    # 메뉴 이름 중복 확인
    if menu_name_ko in visited_data:
      overlapping += 1
      continue # 존재하면 넘어감

    # 새 항목 생성 후 추가
    new_entry = {
        "class": max_class_id + 1,
        "ko": menu_in.get("ko"),
        "en.ROMAN": menu_in.get("en.ROMAN"),
        "en": menu_in.get("en"),
        "ja": menu_in.get("ja"),
        "zh_CN": menu_in.get("zh_CN"),
        "zh_TW": menu_in.get("zh_TW")
    }

    menu_info.append(new_entry)
    max_class_id += 1
    visited_data.add(menu_name_ko)

* 압출 파일 돌며 모든 json 파일 변환하는 함수

In [32]:
# zip 파일 풀어서 파일 옮기기
def move_json(target_folder, tmp_folder):
  for target in target_folder:
    zip_ref.extract(target, tmp_folder) # zip 파일을 임시 폴더로 추출
    extracted_zip_path = os.path.join(tmp_folder, target) # 추출된 zip 파일 경로

    # 내부 zip 파일 열기
    with zipfile.ZipFile(extracted_zip_path, 'r') as inner_zip:
      # 추출된 파일 변환
      for json_file in inner_zip.namelist():
        if json_file.endswith('.json'):
          input_file = inner_zip.open(json_file)
          change_json(input_file)
      print(f'{target} 추출 완료, cnt: {len(menu_info)}, overlap: {overlapping}')

* train, valid 데이터 각각 처리

In [35]:
# 임시 폴더 (압축 해제용)
tmp_folder = '/temp'
os.makedirs(tmp_folder, exist_ok=True)

# 압축 파일 열기
with zipfile.ZipFile(data, 'r') as zip_ref:
      # 이미지 파일 선택
      train_image = [f for f in zip_ref.namelist() if 'labels/train/' in f and f.endswith('.zip')]
      valid_image = [f for f in zip_ref.namelist() if 'labels/val/' in f and f.endswith('.zip')]
    
      # 'train', 'valid' 라벨 파일들 출력
      print(f"Train labels: {train_image}, Valid labels: {valid_image}")
        
      move_json(train_image, tmp_folder)
      move_json(valid_image, tmp_folder)
    
# 결과를 출력 파일로 저장
with open("menu_info.json", "w", encoding='utf-8') as f:
  json.dump(menu_info, f, indent=4, ensure_ascii=False)

Train labels: ['labels/train/TL_01._서울.zip', 'labels/train/TL_02._부산.zip', 'labels/train/TL_03._대구.zip', 'labels/train/TL_04._인천.zip', 'labels/train/TL_05._광주.zip', 'labels/train/TL_06._대전.zip', 'labels/train/TL_07._울산.zip', 'labels/train/TL_08._경기.zip', 'labels/train/TL_09._강원.zip', 'labels/train/TL_10._충청.zip', 'labels/train/TL_11._전라.zip', 'labels/train/TL_12._경상.zip', 'labels/train/TL_13._제주.zip'], Valid labels: ['labels/val/VL_01._서울.zip', 'labels/val/VL_02._부산.zip', 'labels/val/VL_03._대구.zip', 'labels/val/VL_04._인천.zip', 'labels/val/VL_05._광주.zip', 'labels/val/VL_06._대전.zip', 'labels/val/VL_07._울산.zip', 'labels/val/VL_08._경기.zip', 'labels/val/VL_09._강원.zip', 'labels/val/VL_10._충청.zip', 'labels/val/VL_11._전라.zip', 'labels/val/VL_12._경상.zip', 'labels/val/VL_13._제주.zip']
labels/train/TL_01._서울.zip 추출 완료, cnt: 71808, overlap: 210821
labels/train/TL_02._부산.zip 추출 완료, cnt: 88708, overlap: 295699
labels/train/TL_03._대구.zip 추출 완료, cnt: 100748, overlap: 366826
labels/train/TL_04._인천.zip 추